<a href="https://colab.research.google.com/github/willameoliveira/ibot-factory/blob/master/ProjetoImersaoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai
!pip install PyDrive
!pip install PyPDF2

In [14]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata

import PyPDF2

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [24]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# This only needs to be done once per notebook.
# Download a file based on its file ID.

print("Seja bem vindo ao criador de chatbot inteligente. :)")
tipo_negocio = input("Primeiro informe como se identifica seu negócio (ex: empresa, organização ou instituição?): ")
razao_social = input("Informe o nome da sua organização da forma como deseja que apareça no assistente virtual: ")
link_arquivo = input("""Agora informe o link do PDF contido no seu Google Drive que contém as informações dos seus
serviços (CUIDADO: Certifique-se de que todo o conteúdo possa ser publicado na internet e que não há conteúdo ofensivo).
Para isso, basta clicar com o botão direito no arquivo e escolher a opção copiar link para a área de transferência e colar aqui: """)

In [ ]:
#Extraindo o id da URL usando regex
"""
import re

# Extrai o ID da string
regex = r"(?<=id=)([a-zA-Z0-9_-]+)"
match = re.search(regex, link_arquivo)

if match:
  # O ID está no grupo 1
  id = match.group(1)
  print(f"ID extraído: {id}")
else:
  print("ID não encontrado na string")
  """

In [ ]:

from urllib.parse import urlparse

# Analisa a URL
parsed_url = urlparse(link_arquivo)

# O ID está no parâmetro "id"
id = parsed_url.query.split("=")[1].split("&")[0]

print(f"ID extraído: {id}")

In [46]:
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = id
# criando arquivo vazio, apenas com o id do que se deseja acessar
downloaded = drive.CreateFile({'id': file_id})
# baixando o arquivo
downloaded.GetContentFile("document0.pdf")

In [47]:
genai.configure(api_key=userdata.get('API_KEY'))

# Set up the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  # Add logic to read the PDF and return a list of pages here.
  pages = []
  reader = PyPDF2.PdfReader(pathname)
  for p in reader.pages:
    pages.append(p.extract_text())

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

In [ ]:
chat = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages("/content/document0.pdf")#<path>/document0.pdf
  },
  {
    "role": "user",
    "parts": [f"""O arquivo contém informações
    sobre os serviços oferecidos pelo(a) {tipo_negocio} chamado(a) {razao_social}.
    Haja como uma atendente virtual dessa {tipo_negocio} e responda, em
    tom amigável e com uso de emojis, as perguntas que serão feitas apenas
    usando esse documento como base. Antes de começar a responder a primeira
    pergunta, se apresente."""]
  },
])

prompt = input("Digite algo: ")
while prompt != "fim":
  response = chat.send_message(prompt)
  print (f"{razao_social}: {response.text}")
  prompt = input("Digite algo: ")